# PRUEBAS INICIALES
Para ver la eficacia del modelo pre-entrenado, sin haber hecho ningún ajuste sobre los datos particulares. Registramos los resultados en MLflow

In [1]:
from transformers import pipeline
from datasets import load_dataset
import torch
import json
from evaluate import compute_exact,compute_f1,get_raw_scores_by_example,get_raw_scores_by_prediction
from statistics import mean
import mlflow
import pprint
import time
import requests

In [2]:
# Realizaremos las pruebas iniciales con el conjunto de test, cuando se haya concluido el entrenamiento del modelo ajustado (usando train y validation) volveremos a tomar la medida con este conjunto de test. Esto nos servirá para ver si efectivamente el modelo ajustado ha mejorado en algún modo el rendimiento del modelo pre-entrenado.
# Datos y modelo
dataset = load_dataset('..\Dataset\Escrituras','QA',trust_remote_code=True,split='test')

In [3]:
model_QA = 'PlanTL-GOB-ES/roberta-large-bne-sqac' 
# Tamaño de lote para la inferenca
batch_size = 512

In [4]:
SERVIDOR_MLFLOW = 'http://localhost:5000'
# Debo comprobar si está ejecutando el servidor MLflow, en otro caso se demora la ejecución y acaba dando un error
def mlflow_en_ejecucion(url):
    try:
        response = requests.get(url)        
        # Si el servidor está en ejecución, deberíamos recibir un código de estado HTTP 200
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        # Si no se puede establecer una conexión, asumimos que el servidor no está en ejecución
        return False
    
assert mlflow_en_ejecucion(SERVIDOR_MLFLOW), f"El servidor MLflow ({SERVIDOR_MLFLOW}) no está en ejecución. Lance 'mlflow ui' desde el terminal."

In [5]:
# Servidor de seguimiento
mlflow.set_tracking_uri(SERVIDOR_MLFLOW)
mlflow.set_experiment("Valoración inicial")
# Impresión elegante de datos en la terminal
pp = pprint.PrettyPrinter(width=150)

In [6]:
# Un vistazo a los datos que estamos testeando
pp.pprint(dataset.info)

DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'answers': Sequence(feature={'answer_start': Value(dtype='int32', id=None), 'text': Value(dtype='string', id=None)},
                                          length=-1,
                                          id=None),
                      'context': Value(dtype='string', id=None),
                      'id': Value(dtype='string', id=None),
                      'question': Value(dtype='string', id=None)},
            post_processed=None,
            supervised_keys=None,
            task_templates=None,
            builder_name='escrituras',
            dataset_name='escrituras',
            config_name='QA',
            version=0.0.0,
            splits={'test': SplitInfo(name='test', num_bytes=11932583, num_examples=8028, shard_lengths=None, dataset_name='escrituras'),
                    'train': SplitInfo(name='train', num_bytes=38680271, num_examples=256

In [7]:
# Comprobación de si está bien instalado torch y tenemos una GPU
print(torch.cuda.is_available())

True


También he visto que hay varias maneras de realizar las inferencias o consultas sobre el modelo, usando los pipelines. Quiero ver si hay alguna diferencia en el rendimiento. Probaré con distintas formas y cambiaré el tamaño del lote para ver si consigo un rendimiento óptimo.

### Primera prueba

In [ ]:
# Sólo es válida la primera ejecución de esta prueba, ya que HF dataset guarda en caché el map de esta función, si volvemos a llamarla con los mismos parámetros devuelve los resultados de la caché (tardando unos segundos en lugar de algunos minutos)
with mlflow.start_run(run_name="Map sobre test", description="Predicciones usando map sobre el dataset de test"):
    time_0 = time.time()
    # Definición del pipeline y la función de mapeo
    consulta_map =  pipeline("question-answering", model=model_QA, device=0 if torch.cuda.is_available() else None, batch_size=batch_size)
    def realiza_prediccion(ejemplo):
        return {'prediccion': consulta_map({'question':ejemplo['question'], 'context':ejemplo['context']})}
    # Ejecución y cálculo de métricas
    # ejemplos = dataset.map(lambda ejemplo: {'prediccion': consulta_map({'question':ejemplo['question'], 'context':ejemplo['context']})})
    ejemplos = dataset.map(realiza_prediccion)
    exact_scores, f1_scores  = get_raw_scores_by_example(ejemplos)
    f1_mean = mean(f1_scores.values())
    exact_mean = mean(exact_scores.values())
    
    f_time = time.time()-time_0
    mlflow.log_param('batch_size', batch_size)    
    mlflow.log_metric('f1', f1_mean)
    mlflow.log_metric('exact', exact_mean)
    mlflow.log_metric('process_time',f_time)
    print(len(f1_scores), 'f1:', f1_mean)
    print(len(exact_scores), 'exact:', exact_mean)

# Libero la memoria por si afecta a los procesos posteriores
del ejemplos

### Segunda manera

In [8]:

with mlflow.start_run(run_name="Pipeline sobre dataset",description="Predicciones usando columnas del dataset directamente sobre el pipline"):
    time_0 = time.time()
    # Definición del pipeline
    consulta_ds =  pipeline("question-answering", model=model_QA, device=0 if torch.cuda.is_available() else None, batch_size=batch_size)    
    # Ejecución y cálculo de métricas
    predicciones = consulta_ds(question=dataset['question'], context=dataset['context'])
    exact_scores, f1_scores  = get_raw_scores_by_prediction(dataset,predicciones)
    f1_mean = mean(f1_scores.values())
    exact_mean = mean(exact_scores.values())
    
    f_time = time.time()-time_0
    mlflow.log_param('batch_size', batch_size)
    mlflow.log_metric('f1', f1_mean)
    mlflow.log_metric('exact', exact_mean)
    mlflow.log_metric('process_time',f_time)
    print(len(f1_scores), 'f1:', f1_mean)
    print(len(exact_scores), 'exact:', exact_mean)

# Libero la memoria por si afecta a los procesos posteriores
del predicciones

8028 f1: 0.4012164436158013
8028 exact: 0.13789237668161436


### Tercera manera

In [ ]:
with mlflow.start_run(run_name="Pipeline sobre lista pregunta-contexto",description="Predicciones usando una lista de preguntas-contexto extraídas del dataset"):
    time_0 = time.time()
    # Definición del pipeline y el conjunto de datos
    qc_dataset_test = [{'question':q, 'context':c} for q,c in zip(dataset['question'],dataset['context'])]
    consulta_qc = pipeline("question-answering", model=model_QA, device=0 if torch.cuda.is_available() else None, batch_size=batch_size)
    # Ejecución y cálculo de métricas
    predicciones = consulta_qc(qc_dataset_test)
    exact_scores, f1_scores  = get_raw_scores_by_prediction(dataset,predicciones)
    f1_mean = mean(f1_scores.values())
    exact_mean = mean(exact_scores.values())
    
    f_time = time.time()-time_0
    mlflow.log_param('batch_size', batch_size)
    mlflow.log_metric('f1', f1_mean)
    mlflow.log_metric('exact', exact_mean)
    mlflow.log_metric('process_time',f_time)
    print(len(f1_scores), 'f1:', f1_mean)
    print(len(exact_scores), 'exact:', exact_mean)

del predicciones    